## Notebook for making predictions with Normalized train data


In [2]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [3]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [4]:
X_A_train = dm.data_A_obs.iloc[:, 2:-1]
y_A_train = dm.data_A_obs.iloc[:,0]
X_B_train = dm.data_B_obs.iloc[:, 2:-1]
y_B_train = dm.data_B_obs.iloc[:,0]
X_C_train = dm.data_C_obs.iloc[:, 2:-1]
y_C_train = dm.data_C_obs.iloc[:,0]

X_A_val = dm.data_A_es.iloc[:, 2:-1]
y_A_val = dm.data_A_es.iloc[:,0]
X_B_val = dm.data_B_es.iloc[:, 2:-1]
y_B_val = dm.data_B_es.iloc[:,0]

X_C_val = dm.data_C_es.iloc[:, 2:-1]
y_C_val = dm.data_C_es.iloc[:,0]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_b.columns.intersection(X_B_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_c.columns.intersection(X_C_train.columns)]



In [5]:
# Concat

X_train = pd.concat([X_A_train,X_B_train,X_C_train])
y_train = pd.concat([y_A_train,y_B_train,y_C_train])

X_val = pd.concat([X_A_val,X_B_val,X_C_val])
y_val = pd.concat([y_A_val,y_B_val,y_C_val])

#X_test = pd.concat([X_A_test,X_B_test,X_C_test])



## Catboost models

In [6]:
catModel = CatBoostRegressor()

In [7]:
catModel.fit(X_train,y_train)

Learning rate set to 0.102294
0:	learn: 734.8209899	total: 166ms	remaining: 2m 46s
1:	learn: 699.8761517	total: 181ms	remaining: 1m 30s
2:	learn: 670.3467851	total: 196ms	remaining: 1m 5s
3:	learn: 645.3354300	total: 215ms	remaining: 53.6s
4:	learn: 623.9375838	total: 232ms	remaining: 46.2s
5:	learn: 605.3516039	total: 250ms	remaining: 41.4s
6:	learn: 590.1213561	total: 269ms	remaining: 38.1s
7:	learn: 576.3142667	total: 291ms	remaining: 36.1s
8:	learn: 565.0319337	total: 312ms	remaining: 34.4s
9:	learn: 554.8846566	total: 335ms	remaining: 33.2s
10:	learn: 547.4332041	total: 358ms	remaining: 32.2s
11:	learn: 541.1986981	total: 379ms	remaining: 31.2s
12:	learn: 536.1940453	total: 399ms	remaining: 30.3s
13:	learn: 531.6773212	total: 419ms	remaining: 29.5s
14:	learn: 527.7245212	total: 439ms	remaining: 28.8s
15:	learn: 523.7706898	total: 458ms	remaining: 28.1s
16:	learn: 519.4579749	total: 476ms	remaining: 27.5s
17:	learn: 516.9692713	total: 491ms	remaining: 26.8s
18:	learn: 514.2324230	t

In [8]:
score = catModel.score(X_val, y_val)

print(score)

0.4533960587114797


## Preds

In [9]:
pred_A = catModel.predict(X_A_test)#*dm.amplitude[0]
pred_B = catModel.predict(X_B_test)#*dm.amplitude[1]
pred_C = catModel.predict(X_C_test)#*dm.amplitude[2]



In [10]:
df_A = pd.DataFrame()

df_A["prediction"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["prediction"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["prediction"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

# NAME THE FILE 
df.to_csv("sub29.csv")

df.shape

(2160, 1)